In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import pyLDAvis.gensim_models
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import collections

C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\User\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\User\anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\User\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


### LDA 참고: https://wikidocs.net/30708

In [19]:
df= pd.read_csv('kart_title_완.csv',encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24153 entries, 0 to 24152
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   24153 non-null  object
 1   date    24153 non-null  object
dtypes: object(2)
memory usage: 377.5+ KB


In [20]:
#광고, 설문 데이터 제거
values_to_remove = df['title'][0:3]
df = df[~df['title'].isin(values_to_remove)]
# index 초기화
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22782 entries, 0 to 22781
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   22782 non-null  object
 1   date    22782 non-null  object
dtypes: object(2)
memory usage: 356.1+ KB


In [21]:
def remove_non(text):
    # 정규 표현식을 사용하여 한글, 숫자, 알파벳과 띄어쓰기만 남김
    pattern = r'[^\w가-힣\s]+'  # 한글, 숫자, 알파벳, 띄어쓰기를 제외한 모든 문자 제거
    return re.sub(pattern, '', text)

In [22]:
df['clean_doc'] = df['title'].apply(remove_non)
# 길이가 1이하인 단어는 제거 (길이가 짧은 단어 제거)
df['clean_doc'] = df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
# # 전체 단어에 대한 소문자 변환
df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())
#숫자제거
df['clean_doc'] = df['clean_doc'].apply(lambda x: re.sub(r'\d', '', x))

In [23]:
#불용어 출처:https://mr-doosun.tistory.com/24
stop_words = ['app','던파','이것은','너무','분','있음','나만','순서','ㅅㅂ','ㄱㄱ','ㅈㄴ','ㅁㅊ','이제','오늘','게임','ㄹㅇ','ㅋㅋㅋ','ㅋㅋㅋㅋ','ㅋㅋㅋㅋㅋ','지금','근데','존나','아니','씨발','ㅋㅋ','그냥','카드맆','ㅇㅇ','이미지','dc','official','on','on2','on3','on4','on5','on6','아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리','있었어요','있었다','있습니다' '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를', '들면', '예를', '들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수', '없다', '해서는', '안된다', '뿐만', '아니라', '만이', '아니다', '만은', '아니다', '막론하고', '관계없이', '그치지', '않다', '그러나', '그런데', '하지만', '든간에', '논하지', '않다', '따지지', '않다', '설사', '비록', '더라도', '아니면', '만', '못하다', '하는', '편이', '낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이', '외에', '이', '밖에', '하여야', '비로소', '한다면', '몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할', '생각이다', '하려고하다', '이리하여', '그리하여', '그렇게', '함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는', '데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에', '틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해', '말하자면', '뿐이다', '다음에', '반대로', '반대로', '말하자면', '이와', '반대로', '바꾸어서', '말하면', '바꾸어서', '한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에', '가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한', '까닭에', '하기', '때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐떡헐떡', '영차', '여차', '어기여차', '끙끙', '아야', '앗', '아야', '콸콸', '졸졸', '좍좍', '뚝뚝', '주룩주룩', '솨', '우르르', '그래도', '또', '그리고', '바꾸어말하면', '바꾸어서', '말하면', '바꾸어서', '한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에', '가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한', '까닭에', '하기', '때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느', '로써', '까지', '예하면', '했어요', '해요', '함께', '같이', '더불어', '마저', '마저도', '양자', '모두', '습니다', '가까스로', '하려고하다', '즈음하여', '다른', '다른', '방면으로', '해봐요', '습니까', '했어요', '말할것도', '없고', '무릎쓰고', '개의치않고', '하는것만', '못하다', '하는것이', '낫다', '매', '매번', '들', '모', '어느것', '어느', '로써', '갖고말하자면', '어디', '어느쪽', '어느것', '어느해', '어느', '년도', '라', '해도', '언젠가', '어떤것', '어느것', '저기', '저쪽', '저것', '그때', '그럼', '그러면', '요만한걸', '그래', '그때', '저것만큼', '그저', '이르기까지', '할', '지', '않는다면', '만약', '만일', '위에서', '서술한바와같이', '인', '듯하다', '하', '하지', '않는다면', '만약에', '무엇', '무슨', '어느', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '놓', '이거','진짜','시발']
#토큰화
tokenized_doc = df['clean_doc'].apply(lambda x: x.split())
#불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [24]:
#불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 실습 참고)는
#기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다.
#그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서
#다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다.
detokenized_doc = []
for i in range(len(df)):
    
    t = ' '.join(tokenized_doc[i])
    
    detokenized_doc.append(t)
    

df['clean_doc'] = detokenized_doc
df['clean_doc']

0                           이번시즌 아쉬운점
1                  카드립 속도감 조금이나마 느끼는법
2              시즌 미드시즌에 아이스 트랙 업뎃될 같다
3                 다음 핫타임까지 큐돌릴 카창은 개추
4                            나는 신맵 안뜸
                     ...             
22777               리버리 투표 받습니다 시즌 판매
22778                     레나 신음소리 어케됨
22779                  망했다는 기사 슬슬 나오네
22780                        로그인화면 바뀜
22781    템그랑 스쿼드 개편 기념 물폭탄 상황 판단하기tip
Name: clean_doc, Length: 22782, dtype: object

In [25]:
df=df[df['date']!='10.14']
df['date'] = '2023.' + df['date']
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d')
df['month']=df['date'].dt.month

df.info()
# 날짜 범위 설정
start_date_1 = '2023-07-13'
end_date_1 = '2023-08-12'
start_date_2 = '2023-08-13'
end_date_2 = '2023-09-12'
start_date_3 = '2023-09-13'
end_date_3 = '2023-10-12'

# 날짜 범위에 따라 데이터프레임 분할
df_1 = df[(df['date'] >= start_date_1) & (df['date'] <= end_date_1)]
df_2 = df[(df['date'] >= start_date_2) & (df['date'] <= end_date_2)]
df_3 = df[(df['date'] >= start_date_3) & (df['date'] <= end_date_3)]
df = df.reset_index(drop=True)
df_1 = df.reset_index(drop=True)
df_2 = df.reset_index(drop=True)
df_3 = df.reset_index(drop=True)


<class 'pandas.core.frame.DataFrame'>
Index: 22756 entries, 26 to 22781
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      22756 non-null  object        
 1   date       22756 non-null  datetime64[ns]
 2   clean_doc  22756 non-null  object        
 3   month      22756 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 800.0+ KB


In [26]:
vectorizer = TfidfVectorizer(stop_words='english',max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(df['clean_doc'])
X_1 = vectorizer.fit_transform(df_1['clean_doc'])
X_2 = vectorizer.fit_transform(df_2['clean_doc'])
X_3 = vectorizer.fit_transform(df_3['clean_doc'])
# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (22756, 36595)


In [27]:
#토픽 개수
topic_num =10

In [28]:
# n_components : n개의 토픽을 가졌다고 가정
svd_model = TruncatedSVD(n_components=topic_num, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_) 

10

In [29]:
np.shape(svd_model.components_)

(10, 36595)

In [30]:
terms = vectorizer.get_feature_names_out() # 단어 집합.

def get_topics(components, feature_names, n=topic_num):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('템그랑', 0.95116), ('마스터', 0.16086), ('매칭', 0.15722), ('그마', 0.11204), ('시즌', 0.06053), ('스그랑', 0.05554), ('안잡히네', 0.05201), ('골드', 0.03042), ('잡힘', 0.02608), ('주차', 0.02565)]
Topic 2: [('시즌', 0.96741), ('스그랑', 0.0797), ('피트스탑', 0.07432), ('이번', 0.07388), ('업데이트', 0.06285), ('예상', 0.0458), ('카트', 0.03946), ('드리프트', 0.03365), ('카트라이더', 0.03127), ('상점', 0.03026)]
Topic 3: [('매칭', 0.85285), ('스그랑', 0.35494), ('그랑프리', 0.17589), ('안잡히네', 0.10276), ('되네', 0.09993), ('그랑', 0.07667), ('플래그전', 0.05454), ('잡힘', 0.05386), ('매칭이', 0.04743), ('스피드전', 0.04352)]
Topic 4: [('플래그전', 0.95445), ('카트', 0.17293), ('잠수', 0.05456), ('조재윤', 0.04505), ('승리', 0.03572), ('미션', 0.03183), ('카드립', 0.03148), ('이벤트', 0.03129), ('질문', 0.02984), ('니트로', 0.02869)]
Topic 5: [('스그랑', 0.85098), ('카트', 0.25585), ('마스터', 0.12238), ('그마', 0.07237), ('조재윤', 0.05246), ('골드', 0.04749), ('매칭이', 0.04386), ('찍었다', 0.04013), ('왤케', 0.03221), ('원작', 0.02792)]
Topic 6: [('카트', 0.89098), ('조재윤', 0.18526), ('카드립', 0.06542), ('니

In [31]:
# n_components : n개의 토픽을 가졌다고 가정
svd_model = TruncatedSVD(n_components=topic_num, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X_3)
len(svd_model.components_)

10

In [32]:
terms = vectorizer.get_feature_names_out() # 단어 집합.

def get_topics(components, feature_names, n=topic_num):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('템그랑', 0.95116), ('마스터', 0.16086), ('매칭', 0.15722), ('그마', 0.11204), ('시즌', 0.06053), ('스그랑', 0.05554), ('안잡히네', 0.05201), ('골드', 0.03042), ('잡힘', 0.02608), ('주차', 0.02565)]
Topic 2: [('시즌', 0.96741), ('스그랑', 0.0797), ('피트스탑', 0.07432), ('이번', 0.07388), ('업데이트', 0.06285), ('예상', 0.0458), ('카트', 0.03946), ('드리프트', 0.03365), ('카트라이더', 0.03127), ('상점', 0.03026)]
Topic 3: [('매칭', 0.85285), ('스그랑', 0.35494), ('그랑프리', 0.17589), ('안잡히네', 0.10276), ('되네', 0.09993), ('그랑', 0.07667), ('플래그전', 0.05454), ('잡힘', 0.05386), ('매칭이', 0.04743), ('스피드전', 0.04352)]
Topic 4: [('플래그전', 0.95445), ('카트', 0.17293), ('잠수', 0.05456), ('조재윤', 0.04505), ('승리', 0.03572), ('미션', 0.03183), ('카드립', 0.03148), ('이벤트', 0.03129), ('질문', 0.02984), ('니트로', 0.02869)]
Topic 5: [('스그랑', 0.85098), ('카트', 0.25585), ('마스터', 0.12238), ('그마', 0.07237), ('조재윤', 0.05246), ('골드', 0.04749), ('매칭이', 0.04386), ('찍었다', 0.04013), ('왤케', 0.03221), ('원작', 0.02792)]
Topic 6: [('카트', 0.89098), ('조재윤', 0.18526), ('카드립', 0.06542), ('니

In [33]:
documents = [doc.split() for doc in df['clean_doc']]
dictionary = corpora.Dictionary(documents)


corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

## 2) LDA 모델 훈련시키기

In [34]:
import gensim
NUM_TOPICS = 4 # 5개의 토픽, k=5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.023*"템그랑" + 0.020*"플래그전" + 0.009*"카트" + 0.007*"버스트"')
(1, '0.008*"그랑프리" + 0.007*"카드립" + 0.005*"매칭" + 0.005*"이유"')
(2, '0.015*"시즌" + 0.011*"스그랑" + 0.008*"템그랑" + 0.006*"부스터"')
(3, '0.006*"니트로" + 0.004*"속도" + 0.004*"플래그" + 0.004*"베네치아"')


In [35]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

AttributeError: 'list' object has no attribute 'lower'

## 빈도분석
참고:https://brave-greenfrog.tistory.com/40

In [34]:
len(terms)

250239

In [84]:
from konlpy.tag import Okt
okt = Okt()
result =[]
result = [okt.nouns(i) for i in terms] #명사만 추출
final_result= [r for i in result for r in i]
print(final_result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [90]:
##한국어##
korean = pd.Series(final_result).value_counts().head(10)
print("빈도 top 10")
print(korean)

빈도 top 10
임     3041
던파    2443
거     1808
개     1762
함     1729
좀     1703
때     1609
사람    1388
던     1373
캐릭    1259
dtype: int64
